In [80]:
from datetime import datetime, timedelta
import mediapipe as mp
import pandas as pd
import numpy as np
import random
import cvzone
import time
import cv2
import os

In [81]:
def HandTracker():
    """
    Function that returns hand tracker object
    """
    mp_drawings = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    hand_tracker = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.8, min_tracking_confidence=0.5)
    return hand_tracker

In [82]:
def drawLandMarks(results, image):
    """
    Function that draws land marks
    """
    mp_drawings = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
            mp_drawings.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
            # Joints Color
            mp_drawings.DrawingSpec(color=(121, 22, 76), thickness=4, circle_radius=6),
            # Line Color
            mp_drawings.DrawingSpec(color=(121, 44, 250), thickness=10, circle_radius=2))
    return image

In [83]:
INDEX_UP = 8
INDEX_DOWN = 6
MIDDLE_FINGER_UP = 12
MIDDLE_FINGER_DOWN = 10
RING_FINGER_UP = 16
RING_FINGER_DOWN = 14
PINKY_UP = 20
PINKY_DOWN = 18
THUMB_UP = 4
THUMB_DOWN = 2

### For Index Finger

In [84]:
def checkIndex(results):
    condition = False
    if results.multi_hand_landmarks:
        UP = results.multi_hand_landmarks[0].landmark[INDEX_UP].y
        DOWN = results.multi_hand_landmarks[0].landmark[INDEX_DOWN].y

        if UP < DOWN:
            condition = True
        
        if UP > DOWN:
            condition = False
    return condition

### For Middle Finger

In [85]:
def checkMiddle(results):
    condition = False
    if results.multi_hand_landmarks:
        UP = results.multi_hand_landmarks[0].landmark[MIDDLE_FINGER_UP].y
        DOWN = results.multi_hand_landmarks[0].landmark[MIDDLE_FINGER_DOWN].y

        if UP < DOWN:
            condition = True
        
        if UP > DOWN:
            condition = False
    return condition

### For Ring Finger

In [86]:
def checkRing(results):
    condition = False
    if results.multi_hand_landmarks:
        UP = results.multi_hand_landmarks[0].landmark[RING_FINGER_UP].y
        DOWN = results.multi_hand_landmarks[0].landmark[RING_FINGER_DOWN].y

        if UP < DOWN:
            condition = True
        
        if UP > DOWN:
            condition = False
    return condition

### For Pinky Finger

In [87]:
def checkPinky(results):
    condition = False
    if results.multi_hand_landmarks:
        UP = results.multi_hand_landmarks[0].landmark[PINKY_UP].y
        DOWN = results.multi_hand_landmarks[0].landmark[PINKY_DOWN].y

        if UP < DOWN:
            condition = True
        
        if UP > DOWN:
            condition = False
    return condition

### For Thumb

In [88]:
def checkThumb(results):
    condition = False
    if results.multi_hand_landmarks:
        UP = results.multi_hand_landmarks[0].landmark[THUMB_UP].x
        DOWN = results.multi_hand_landmarks[0].landmark[THUMB_DOWN].x

        if UP < DOWN:
            condition = True
        
        if UP > DOWN:
            condition = False
    return condition

### Image Loader

In [89]:
def load_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (480, 480))
    return image

### Text

In [90]:
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50) 
fontScale = 2 
color = (255, 255, 0)
thickness = 2

### Main Function

In [91]:
hand_tracker = HandTracker()
stream = cv2.VideoCapture(0)
counter = 50
counterCX = 0
images = ['rock.png', 'paper.png', 'scissors.png']
string = ""
sign = "Select sign in "
r_image = None
ai_points = 0
computer_points = 0
r_tag = ""
window_name = "Rock, Paper and Scissors Game using AI"


cv2.namedWindow(window_name, cv2.WINDOW_NORMAL);
cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN);
cv2.setWindowProperty(window_name, cv2.WND_PROP_TOPMOST, 1)


while stream.isOpened():
    ret, frame = stream.read()
    
    image = cv2.flip(frame, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hand_tracker.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.blur(image, (55, 55))
    image = drawLandMarks(results, image)
    index = checkIndex(results)
    middle = checkMiddle(results)
    ring = checkRing(results)
    pinky = checkPinky(results)
    thumb = checkThumb(results)
    if results.multi_hand_landmarks:
        if index and middle and not ring and not pinky and not thumb:
            string = "Scissors"
        elif index and middle and ring and pinky and thumb:
            string = "Paper"
        elif not index and not middle and not ring and not pinky and not thumb:
            string = "Rock"
        else:
            string = "Please correct your sign."
    else:
        string = "No hand present."

    image = cv2.putText(image, "AI", (20, 420), font, fontScale, (0, 0, 255), 3, cv2.LINE_AA)
    
    r_image = random.choice(images)
    g_image = load_image(r_image)
    r_tag = r_image.split(".")[0]
    try:
        image = cv2.putText(image, string, org, font, 1, color, thickness, cv2.LINE_AA)
        g_image = cv2.putText(g_image, r_tag, org, font, 1, color, thickness, cv2.LINE_AA)
        g_image = cv2.putText(g_image, "Computer", (20, 420), font, fontScale, (0, 0, 255), 3, cv2.LINE_AA)
        compound = cvzone.stackImages([image, g_image], 2, 1)
    except:
        canvas = np.zeros([480, 480],dtype=np.uint8)
        canvas.fill(255)
        canvas = cv2.putText(canvas, r_tag, org, font, 1, color, thickness, cv2.LINE_AA)
        canvas = cv2.putText(canvas, "Computer", (20, 420), font, fontScale, (0, 0, 255), 3, cv2.LINE_AA)
        compound = cvzone.stackImages([image, canvas], 2, 1)


    if counter == 0:
        time.sleep(3)
        try:
            if (string.lower() == "paper") and (r_tag == "rock"):
                ai_points = ai_points + 1
            
            if (string.lower() == "rock") and (r_tag == "scissors"):
                ai_points = ai_points + 1

            if (string.lower() == "scissors") and (r_tag == "paper"):
                ai_points = ai_points + 1

            if (string.lower() == "paper") and (r_tag == "scissors"):
                computer_points = computer_points + 1

            if (string.lower() == "rock") and (r_tag == "paper"):
                computer_points = computer_points + 1

            if (string.lower() == "scissors") and (r_tag == "rock"):
                computer_points = computer_points + 1
            
            print(string, r_image.split(".")[0], ai_points, computer_points)
        except:
            print(None, None)
        counter = 50
    else:
        sign = "Select sign in "
        sign = sign+str(counter)+" seconds."
        counter = counter - 1
    
    compound = cv2.putText(compound, sign, (150, 420), font, 1, (121, 22, 76), 2, cv2.LINE_AA)
    compound = cv2.putText(compound, f"AI Points: {str(ai_points)}", (5, 150), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    compound = cv2.putText(compound, f"Computer Points: {str(computer_points)}", (5, 200), font, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.imshow(window_name, compound)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

stream.release()
cv2.destroyAllWindows()

Paper paper 0 0
Scissors rock 0 1
Rock rock 0 1
Rock rock 0 1
Paper rock 1 1
Paper scissors 1 2


In [79]:
stream.release()
cv2.destroyAllWindows()